In [1]:
import pandas as pd
import cfb_ou as cf
import numpy as np
import cfb_model as cm

### Start by updating the schedule in R, the updated file will be exported here. 

**_Preparing data for Visualization in `

In [2]:
def get_gametimes(week):
    data = pd.read_csv('cfb_hist.csv')
    data = data.drop_duplicates(subset='game_id')
    data = data.sort_values(by=['season', 'week'], ascending=[True, True])
    data = data[(data['season'].isin([2024]))]
    data = data[(data['week'].isin([week]))]
    data = data.rename(columns={'start_date':'gameday'})
    data['gameday'] = pd.to_datetime(data['gameday'])
    data['gameday'] = data['gameday'].dt.tz_convert('America/Los_Angeles')
    data['time'] = data['gameday'].dt.strftime('%H:%M:%S')
    data['gameday'] = data['gameday'].dt.date
    data['gameday'] = pd.to_datetime(data['gameday'])

    data = data[['home_team', 'away_team', 'gameday', 'time']]


    return data

### Run the Model

**_Need to update the data for the current week in `CFBFastR` file_**

In [ ]:
new_df = cf.get_over_under()

In [ ]:
new_df.to_csv('cfb_ou_full.csv')

In [2]:
data = cm.model_output(8) #model_output(week), takes about 15 Minutes

In [ ]:
data.to_csv('week8_full.csv') #Save it as csv for easy access

In [ ]:
data = pd.read_csv('week8_full.csv')

In [3]:
raw = get_gametimes(8)

In [126]:
merged_data = pd.merge(
    data, 
    raw[['home_team', 'away_team', 'gameday', 'time']],
    on=['home_team', 'away_team', 'gameday'], 
    how='left'
)

In [ ]:
merged_data.to_csv('updated_times.csv') # Go into excel and update the times

In [40]:
data = pd.read_csv('updated_times.csv', index_col=0) #Import updated times 

In [41]:
data['gameday'] = pd.to_datetime(data['gameday'])

In [42]:
len(data) #Amount of games

59

In [43]:
thursday = data[data['gameday'] == '2024-10-17'] #Insert Dates

In [44]:
thursday.head()

,home_team,away_team,total_line,TPAPG,TPPG,gameday,prediction,pred-actual,prob_correct_pred,time
12,UTEP,Florida International,47.0,63.00,43.00,2024-10-17,49.5,2.5,0.860,NaN
17,Marshall,Georgia State,51.5,56.73,50.57,2024-10-17,50.5,-1.0,0.273,16:00:00
19,Virginia Tech,Boston College,48.5,39.83,57.67,2024-10-17,47.0,-1.5,0.211,16:30:00


In [45]:
friday = data[data['gameday'] == '2024-10-18'] #Insert Dates

In [46]:
friday

,home_team,away_team,total_line,TPAPG,TPPG,gameday,prediction,pred-actual,prob_correct_pred,time
3,Purdue,Oregon,60.0,59.40,60.27,2024-10-18,57.5,-2.5,0.667,5:00:00
21,Duke,Florida State,42.5,42.83,41.17,2024-10-18,40.5,-2.0,0.086,16:00:00
31,Nevada,Fresno State,50.0,54.64,56.12,2024-10-18,52.0,2.0,0.740,19:30:00
34,BYU,Oklahoma State,52.5,44.17,63.17,2024-10-18,51.5,-1.0,0.373,19:15:00


In [47]:
saturday = data[data['gameday'] == '2024-10-19'] #Insert Dates

In [48]:
saturday = saturday.sort_values(by=['time'], ascending=True)

In [49]:
saturday.head()

,home_team,away_team,total_line,TPAPG,TPPG,gameday,prediction,pred-actual,prob_correct_pred,time
28,Temple,Tulsa,53.0,73.17,43.17,2024-10-19,53.0,0.0,0.000,11:00:00
54,Eastern Michigan,Central Michigan,54.0,53.83,58.50,2024-10-19,53.0,-1.0,0.463,11:00:00
33,Tennessee,Alabama,56.0,31.50,83.83,2024-10-19,54.0,-2.0,0.357,12:30:00
41,Northern Illinois,Toledo,43.0,38.17,58.67,2024-10-19,46.0,3.0,1.000,12:30:00
30,Navy,Charlotte,55.0,51.87,67.60,2024-10-19,57.5,2.5,0.530,12:30:00


In [50]:
len(saturday)

48

In [51]:
48/3

16.0

### Split Saturday Games into Multiple DF's

In [52]:
saturday_one = saturday.head(16)

In [53]:
saturday_two = saturday.iloc[16:32]

In [54]:
saturday_three = saturday.tail(16)

In [55]:
thursday.to_csv('thursday.csv')
friday.to_csv('friday.csv')
saturday_one.to_csv('saturday_one.csv')
saturday_two.to_csv('saturday_two.csv')
saturday_three.to_csv('saturday_three.csv')

In [56]:
check = len(saturday_one) + len(saturday_two) + len(saturday_three)

In [57]:
check #Should be the same as len(saturday)

48

In [58]:
data.head()

,home_team,away_team,total_line,TPAPG,TPPG,gameday,prediction,pred-actual,prob_correct_pred,time
0,Michigan State,Iowa,39.5,42.20,46.83,2024-10-19,43.0,3.5,0.900,16:30:00
1,Maryland,USC,57.0,45.17,60.17,2024-10-19,50.0,-7.0,0.422,13:00:00
2,Northwestern,Wisconsin,41.5,39.00,51.67,2024-10-19,44.0,2.5,1.000,9:00:00
3,Purdue,Oregon,60.0,59.40,60.27,2024-10-18,57.5,-2.5,0.667,5:00:00
4,South Alabama,Troy,53.0,55.00,57.17,2024-10-15,53.0,0.0,0.000,16:30:00


### Make DF's for Games With Confidence > 50%

In [59]:
my_bets = data[(data['prob_correct_pred'] > 0.50) &
                (data['gameday'].isin(['2024-10-19', '2024-10-18']))]

In [60]:
my_bets['time'] = pd.to_datetime(my_bets['time'], format='%H:%M:%S').dt.time
my_bets = my_bets.sort_values(by=['gameday','time'], ascending=[True, True])

In [61]:
my_bets

,home_team,away_team,total_line,TPAPG,TPPG,gameday,prediction,pred-actual,prob_correct_pred,time
3,Purdue,Oregon,60.0,59.40,60.27,2024-10-18,57.5,-2.5,0.667,05:00:00
31,Nevada,Fresno State,50.0,54.64,56.12,2024-10-18,52.0,2.0,0.740,19:30:00
2,Northwestern,Wisconsin,41.5,39.00,51.67,2024-10-19,44.0,2.5,1.000,09:00:00
5,Rutgers,UCLA,40.0,48.67,40.17,2024-10-19,42.5,2.5,1.000,09:00:00
9,UConn,Wake Forest,55.5,59.87,60.23,2024-10-19,56.0,0.5,0.600,09:00:00
14,Louisville,Miami,59.5,38.50,83.83,2024-10-19,58.0,-1.5,0.534,09:00:00
47,Coastal Carolina,Louisiana,57.5,56.37,66.17,2024-10-19,58.0,0.5,0.516,09:00:00
36,Oklahoma,South Carolina,41.5,39.33,51.83,2024-10-19,43.5,2.0,1.000,09:45:00
11,California,NC State,47.0,50.12,50.45,2024-10-19,48.0,1.0,0.860,12:30:00
27,Tulane,Rice,52.5,48.33,68.00,2024-10-19,56.0,3.5,0.671,12:30:00


In [62]:
len(my_bets)

22

In [63]:
my_bets_one = my_bets.head(12)

In [64]:
my_bets_one.to_clipboard()

In [65]:
my_bets_two = my_bets.tail(10)

In [66]:
my_bets_two

,home_team,away_team,total_line,TPAPG,TPPG,gameday,prediction,pred-actual,prob_correct_pred,time
41,Northern Illinois,Toledo,43.0,38.17,58.67,2024-10-19,46.0,3.0,1.000,12:30:00
53,Buffalo,Western Michigan,47.5,57.33,48.67,2024-10-19,49.5,2.0,0.780,12:30:00
16,Arizona,Colorado,58.0,50.83,56.17,2024-10-19,50.5,-7.5,0.558,13:00:00
23,Texas Tech,Baylor,55.5,56.33,67.33,2024-10-19,58.5,3.0,0.600,13:00:00
57,Utah State,New Mexico,78.0,91.27,58.13,2024-10-19,69.0,-9.0,0.750,13:00:00
58,Vanderbilt,Ball State,57.5,69.50,61.67,2024-10-19,63.0,5.5,0.516,16:00:00
0,Michigan State,Iowa,39.5,42.20,46.83,2024-10-19,43.0,3.5,0.900,16:30:00
26,Memphis,North Texas,66.5,48.83,72.00,2024-10-19,58.0,-8.5,0.765,16:30:00
15,Stanford,SMU,53.5,56.23,57.53,2024-10-19,54.5,1.0,0.620,17:00:00
24,Utah,TCU,49.5,47.83,62.17,2024-10-19,52.0,2.5,0.769,19:30:00


### Export My_Bets to CSV

In [67]:
my_bets_one.to_csv('my_bets_one.csv')
my_bets_two.to_csv('my_bets_two.csv')